# 🖼️ 1. Datensatz vorbereiten

Jede Gruppe erhält einen Ordner mit **Rohbildern**. Diese sollen in [Makesense.ai](https://www.makesense.ai) annotiert werden.

## 🔧 Schritt 1: Annotation mit Makesense.ai
0. Ladet die Rohbilder mit eurer Gruppennummer unter folgender [URL](https://databox.bonn.de/public/upload-shares/YhXlVGe57thQAg8WUjVlZFNK6z4LORiq) runter
1. Öffnet [https://www.makesense.ai](https://www.makesense.ai)
2. Bilder hochladen
3. Modus: **Segmentation**
4. Klasse: z. B. `baum`, `überdachung`
5. Export: Format `Pascal VOC Segmentation (PNG masks)`

👉 Jetzt bitte eure annotierten Masken und Bilder wieder hochladen. Verwende folgenden Platzhalter:


In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os

# 📁 Pfade anpassen
images_dir = "gruppe1/images"
annotations_file = "gruppe1/annotations.json"  # COCO JSON-Datei

# 🐒 Lade COCO-Annotationen
coco = COCO(annotations_file)
image_ids = coco.getImgIds()

# 🔁 Durch einige Bilder iterieren
for img_id in image_ids[:3]:  # Zeige 3 Beispielbilder
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(images_dir, img_info['file_name'])

    image = Image.open(img_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    ax = plt.gca()

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    
    for ann in anns:
        if 'segmentation' in ann and isinstance(ann['segmentation'], list):
            for seg in ann['segmentation']:
                poly = np.array(seg).reshape((len(seg) // 2, 2))
                patch = patches.Polygon(poly, fill=False, edgecolor='red', linewidth=2)
                ax.add_patch(patch)

    plt.title(img_info['file_name'])
    plt.axis('off')
    plt.show()
